[@LorenaABarba](https://twitter.com/LorenaABarba)

[@ggruszczynski](https://github.com/ggruszczynski)

12 steps to Navier–Stokes
======
***

Hello! Welcome to the **12 steps to Navier–Stokes**. This is a practical module that is used in the beginning of an interactive Computational Fluid Dynamics (CFD) course taught by [Prof. Lorena Barba](http://lorenabarba.com) since Spring 2009 at Boston University. The course assumes only basic programming knowledge (in any language) and of course some foundation in partial differential equations and fluid mechanics. The practical module was inspired by the ideas of Dr. Rio Yokota, who was a post-doc in Barba's lab, and has been refined by Prof. Barba and her students over several semesters teaching the course. The course is taught entirely using Python and students who don't know Python just learn as we work through the module.

This [Jupyter notebook](https://jupyter-notebook.readthedocs.io/en/stable/) will lead you through the first step of programming your own Navier–Stokes solver in Python from the ground up.  We're going to dive right in.  Don't worry if you don't understand everything that's happening at first, we'll cover it in detail as we move forward and you can support your learning with the videos of [Prof. Barba's lectures on YouTube](http://www.youtube.com/playlist?list=PL30F4C5ABCE62CB61).

For best results, after you follow this notebook, prepare your own code for Step 1, either as a Python script or in a clean Jupyter notebook.

To execute this Notebook, we assume you have invoked the notebook server using: `jupyter notebook`.

Step 1: 1-D Linear Convection
-----
***

The 1-D Linear Convection equation is the simplest, most basic model that can be used to learn something about CFD. It is surprising that this little equation can teach us so much! Here it is:

$$\frac{\partial u}{\partial t} + c \frac{\partial u}{\partial x} = 0$$

With given initial conditions (understood as a *wave*), the equation represents the propagation of that initial *wave* with speed $c$, without change of shape. Let the initial condition be $u(x,0)=u_0(x)$. Then the exact solution of the equation is $u(x,t)=u_0(x-ct)$.

We discretize this equation in both space and time, using the Forward Difference scheme for the time derivative and the Backward Difference scheme for the space derivative. Consider discretizing the spatial coordinate $x$ into points that we index from $i=0$ to $N$, and stepping in discrete time intervals of size $\Delta t$.

From the definition of a derivative (and simply removing the limit), we know that:

$$\frac{\partial u}{\partial x}\approx \frac{u(x+\Delta x)-u(x)}{\Delta x}$$


## Explicit, one-sided (backward) FD 

Our discrete equation, then, is:

$$\frac{u_i^{n+1}-u_i^n}{\Delta t} + c \frac{u_i^n - u_{i-1}^n}{\Delta x} = 0 $$

Where $n$ and $n+1$ are two consecutive steps in time, while $i-1$ and $i$ are two neighboring points of the discretized $x$ coordinate. If there are given initial conditions, then the only unknown in this discretization is $u_i^{n+1}$.  We can solve for our unknown to get an equation that allows us to advance in time, as follows:

$$u_i^{n+1} = u_i^n - c \frac{\Delta t}{\Delta x}(u_i^n-u_{i-1}^n)$$

Now let's try implementing this in Python.  

We'll start by importing a few libraries to help us out.

* `numpy` is a library that provides a bunch of useful matrix operations akin to MATLAB
* `matplotlib` is a 2D plotting library that we will use to plot our results
* `time` and `sys` provide basic timing functions that we'll use to slow down animations for viewing

In [ ]:
# Remember: comments in python are denoted by the pound sign
import numpy                       #here we load numpy
from matplotlib import pyplot      #here we load matplotlib
import time, sys                   #and load some utilities


In [ ]:
#this makes matplotlib plots appear in the notebook (instead of a separate window)
%matplotlib inline                  

Now let's define a few variables; we want to define an evenly spaced grid of points within a spatial domain that is 2 units of length wide, i.e., $x_i\in(0,2)$.  We'll define a variable `nx`, which will be the number of grid points we want and `dx` will be the distance between any pair of adjacent grid points.  

In [ ]:
# Explicit one sided FD - observe numerical diffusion
nx = 41    # try changing this number from 41 to 81 and Run All ... what happens?
nt = 15    #nt is the number of timesteps we want to calculate
dt = .025  #dt is the amount of time each timestep covers (delta t)
c = 1      #assume wavespeed of c = 1

# try the following settings (for central FD)
nx *= 10
nt *= 500
dt *= 0.001

domain_length = 2
dx = domain_length / (nx-1)

We also need to set up our initial conditions. The initial velocity $u_0$ is given as 
$u = 2$ in the interval $0.5 \leq x \leq 1$  and $u = 1$ everywhere else in $(0,2)$ (i.e., a hat function).

Here, we use the function `ones()` defining a `numpy` array which is `nx` elements long with every value equal to 1.

In [ ]:

xspace = numpy.linspace(0, domain_length, nx)
u_IC = numpy.ones(nx)      #numpy function ones()
u_IC[int(.5 / dx):int(1 / dx + 1)] = 2  #setting u = 2 between 0.5 and 1 as per our I.C.s
# int((nx-1)/4)
# int(nx/2 + 1)

# different IC
# xspace=numpy.linspace(0, domain_length, num=nx, endpoint=True)
# u_IC = numpy.tanh(xspace) 

# x_mean = 0.5
# L2 = (xspace-x_mean)**2
# C0 = 1 # initial concentration
# Sigma2 = 0.05 # initial variance
# u_IC = C0*numpy.exp(-L2/(2*Sigma2))

# u_IC = numpy.sin(xspace*(2*numpy.pi/max(xspace)))
pyplot.plot(xspace, u_IC)
# print(u_IC)

Now let's take a look at those initial conditions using a Matplotlib plot.  We've imported the `matplotlib` plotting library `pyplot` and the plotting function is called `plot`, so we'll call `pyplot.plot`. To learn about the myriad possibilities of Matplotlib, explore the [Gallery](http://matplotlib.org/gallery.html) of example plots.

Here, we use the syntax for a simple 2D plot: `plot(x,y)`, where the `x` values are evenly distributed grid points:

In [ ]:
pyplot.plot(xspace, u_IC)

Why doesn't the hat function have perfectly straight sides? Think for a bit.

Now it's time to implement the discretization of the convection equation using a finite-difference scheme.  

For every element of our array `u`, we need to perform the operation $u_i^{n+1} = u_i^n - c \frac{\Delta t}{\Delta x}(u_i^n-u_{i-1}^n)$

We'll store the result in a new (temporary) array `un`, which will be the solution $u$ for the next time-step.  We will repeat this operation for as many time-steps as we specify and then we can see how far the wave has convected.  

We first initialize our placeholder array `un` to hold the values we calculate for the $n+1$ timestep, using once again the NumPy function `ones()`.

Then, we may think we have two iterative operations: one in space and one in time (we'll learn differently later), so we'll start by nesting one loop inside the other. Note the use of the nifty `range()` function. When we write: `for i in range(1,nx)` we will iterate through the `u` array, but we'll be skipping the first element (the zero-th element).  *Why?*

In [ ]:
# backward FD

un_bfd = u_IC.copy()
u = u_IC.copy()

for n in range(nt):  #loop for values of n from 0 to nt, so it will run nt times
    un_bfd = u.copy() ##copy the existing values of u into un
    # for i in range(1, nx): ## you can try commenting this line and...
    for i in range(nx): ## ... uncommenting this line and see what happens!
        u[i] = un_bfd[i] - c * dt / dx * (un_bfd[i] - un_bfd[i-1]) # backward FD

pyplot.plot(xspace, u)

**Note**—We will learn later that the code as written above is quite inefficient, and there are better ways to write this, Python-style. But let's carry on.

Now let's try plotting our `u` array after advancing in time.

OK! So our hat function has definitely moved to the right, but it's no longer a hat.  **What's going on?**

## Explicit, central FD - part I

Let us try a more accurate discretization stencil, namely central finite difference:

$$\frac{\partial u}{\partial x}\approx \frac{u(x+\Delta x)-u(x-\Delta x)}{2\Delta x}$$

Now, the discrete scheme can be expressed as:

$$\frac{u_i^{n+1}-u_i^n}{\Delta t} + c \frac{u_{i+1}^n - u_{i-1}^n}{2\Delta x} = 0 $$

$$u_i^{n+1} = u_i^n - c \frac{\Delta t}{2 \Delta x}(u_{i+1}^n-u_{i-1}^n) =  \beta u_{i-1}^n + u_i^n  - \beta u_{i+1}^n$$

where

$$ \beta = c \frac{\Delta t}{2 \Delta x} $$

In [ ]:
# explicit central FD
un_ecfd = numpy.ones(nx) #initialize a temporary array - central finite difference
un_ecfd = u_IC.copy()
u = u_IC.copy()  

for n in range(nt):  #loop for values of n from 0 to nt, so it will run nt times
    un_ecfd = u.copy() ##copy the existing values of u into un
    for i in range(1, nx-1): ## you can try commenting this line and...
    # for i in range(nx): ## ... uncommenting this line and see what happens!
    # remember to use un_ecfd instead of un_bfd 
        # u[i] = ...  # central FD 

pyplot.plot(xspace, u);

## Explicit, central FD - part II

Notice, that the scheme can be formulated as:

$$
\boldsymbol{u}^{n+1} = \mathbb{A} \boldsymbol{u}^n 
$$


Observe, that $ \mathbb{A} $ have a tridiagonal structure:

$$
\begin{bmatrix} 
A_{0,0} & A_{0,1} &  & \\
\beta  & 1 & -\beta \\
0 & \beta  & 1 & -\beta \\
0 & 0 & \beta & 1 & -\beta \\
... \\
... \\
0 & 0 & 0 & 0 & 0 & \beta & 1 & -\beta \\
0 & 0 & 0 & 0 & 0 & 0 & A_{n,n-1}  & A_{n,n}\\
\end{bmatrix}
\quad
$$

**Hint:**

For simplification, fill the corners of the matrix using:
* forward FD for $A_{0,0}$ and $A_{0,1}$

$$\frac{u_i^{n+1}-u_i^n}{\Delta t} + c \frac{u_{i+}^n - u_{i}^n}{\Delta x} = 0 $$

$$u_i^{n+1} = u_i^n - c \frac{\Delta t}{\Delta x}(u_{i+1}^n-u_{i}^n) = u_i^n (1+\lambda) - \lambda u_{i+1}^n$$


* backward FD for $A_{n,n}$ and $A_{n,n-1}$
$$\frac{u_i^{n+1}-u_i^n}{\Delta t} + c \frac{u_i^n - u_{i-1}^n}{\Delta x} = 0 $$

$$u_i^{n+1} = u_i^n - c \frac{\Delta t}{\Delta x}(u_i^n-u_{i-1}^n) = u_i^n (1-\lambda) + \lambda u_{i-1}^n$$

where $\lambda = c \frac{\Delta t}{\Delta x}$


In [ ]:
# explicit central FD - matrix approach
un_ecfd = numpy.ones(nx) #initialize a temporary array - central finite difference
un_ecfd = u_IC.copy()
u = u_IC.copy()  


A = numpy.zeros((nx, nx))      
# Beta_central_FD = ...
# Beta_one_side_FD = ...

last_index_in_matrix = nx -1 
# the BC  - use one sided FD            
# A[0, 0] = ...   # forward FD
# A[0, 1] = ...   # forward FD 
# A[last_index_in_matrix, last_index_in_matrix-1] = ...   # backward FD 
# A[last_index_in_matrix, last_index_in_matrix] = ...     # backward FD 
for i in range(1, last_index_in_matrix): 
    # A[i, i-1] = ...         # left of the diagonal
    # A[i, i] = ...           # the diagonal
    # A[i, i+1] = ...         # right of the diagonal   

print(f"determinant A: {numpy.linalg.det(A)}") # determinant =< 1 for stability

for n in range(nt):  #loop for values of n from 0 to nt, so it will run nt times
    un_ecfd = A@un_ecfd

pyplot.plot(xspace, un_ecfd);


Although the central FD shall be more accurate (formally $2nd$ order), the scheme turns out to be divergent in longer runs. 
What information provides the determinant of $\mathbb{A}$?



## implicit, central FD

In the next attempt, let us try an implicit, central FD.

The values from the future are used to construct the stencil for the spatial derivative, which lead to implicitness.

The scheme reads:

$$\frac{u_i^{n+1}-u_i^n}{\Delta t} + c \frac{u_{i+1}^{n+1} - u_{i-1}^{n+1}}{2\Delta x} = 0 $$

$$u_i^{n+1} + c \frac{\Delta t}{2 \Delta x}(u_{i+1}^{n+1}-u_{i-1}^{n+1}) = u_i^n$$

$$ - \beta u_{i-1}^{n+1} + u_i^{n+1} + \beta u_{i+1}^{n+1} = u_i^n$$

A system of equations has to be solved to find the subsequent time step.

$$
\mathbb{A} \boldsymbol{u}^{n+1} = \boldsymbol{u}^n 
$$


Again, the $ \mathbb{A} $ matrix have a tridiagonal structure (a bit different than in the explicit case):

$$
\begin{bmatrix} 
A_{0,0} & A_{0,1} &  & \\
-\beta  & 1 & \beta \\
0 & -\beta  & 1 & \beta \\
0 & 0 & -\beta & 1 & \beta \\
... \\
... \\
0 & 0 & 0 & 0 & 0 & -\beta & 1 & \beta \\
0 & 0 & 0 & 0 & 0 & 0 & A_{n,n-1}  & A_{n,n}\\
\end{bmatrix}
\quad
$$


**Hint:**

For simplification, fill the corners of the matrix using:
* forward implicit FD for $A_{0,0}$ and $A_{0,1}$

$$\frac{u_i^{n+1}-u_i^n}{\Delta t} + c \frac{u_{i+1}^{n+1} - u_{i}^{n+1}}{\Delta x} = 0 $$

$$ u_i^{n+1} + c \frac{\Delta t}{\Delta x}(u_{i+1}^{n+1}-u_{i}^{n+1}) =  u_i^n $$

$$ u_i^{n+1} (1-\lambda) + \lambda u_{i+1}^{n+1} $$

* backward implicit FD for $A_{n,n}$ and $A_{n,n-1}$
$$\frac{u_i^{n+1}-u_i^n}{\Delta t} + c \frac{u_i^{n+1} - u_{i-1}^{n+1}}{\Delta x} = 0 $$

$$ u_i^{n+1} + c \frac{\Delta t}{\Delta x}(u_i^{n+1}-u_{i-1}^{n+1}) = u_i^n $$

$$ u_i^{n+1} (1+\lambda) - \lambda u_{i-1}^{n+1} = u_i^n $$

where $\lambda = c \frac{\Delta t}{\Delta x}$


In [ ]:
# implicit central FD
import numpy as np
np.set_printoptions(precision=3, suppress=True)
un_icfd = numpy.ones(nx) #initialize a temporary array - central finite difference
un_icfd = u_IC.copy()
u = u_IC.copy()  

A = numpy.zeros((nx, nx))
                
# Beta_central_FD = ...
# Beta_one_side_FD = ...

last_index_in_matrix = nx -1 
# the BC  - use one sided FD            
# A[0, 0] = ...    # forward FD
# A[0, 1] = ...    # forward FD 
# A[last_index_in_matrix, last_index_in_matrix-1] = ...   # backward FD 
# A[last_index_in_matrix, last_index_in_matrix] =  ...    # backward FD 
for i in range(1, last_index_in_matrix): 
    # A[i, i-1] = ...    # left of the diagonal
    # A[i, i] = ...      # the diagonal
    # A[i, i+1] = ...    # right of the diagonal   

    

A_inv = ... # use numpy to invert the matrix

print(f"determinant A_inv: {numpy.linalg.det(A_inv)}") # determinant =< 1 for stability

for n in range(nt):  #loop for values of n from 0 to nt, so it will run nt times
    un_icfd = A_inv@un_icfd
    # alternative way of doing the same:
    #     un_icfd = np.dot(A_inv,un_icfd)
    #     b = un_icfd.copy() 
    #     un_icfd = np.linalg.solve(A, b) # u(t+1)
                
pyplot.plot(xspace, un_icfd)

## Questions

* discuss the differences between first order (one sided, forward/backward FD) and second order stencils (central FD). 
    * How do they influence numerical diffusion and dispersion?
    * How do they influence stability of the scheme?
    * The one-sided FD is also refereed as an upwind method. Can the forward/backward FD be used interchangably?  
    * Is it possible to fill the corners of the $\mathbb{A}$ matrix using second order stencil?

* Would the hat function get diffused/disturbed if advected by exactly one cell per time step?

* Why artefacts in second order (or higher) schemes exhibits as wiggles instead of just smaller diffusion?

* Is there (in general) a tradeoff between accuracy and stability of the scheme?

* Is the $\mathbb{A}$ matrix always a constant one? 

* (Extra): Can one take the best of both worlds, i.e. mix first and second order schemes to limit the unphysical wiggles (oscilations)?


Learn More
-----
***

For a more thorough explanation of the finite-difference method, including topics like the truncation error, order of convergence and other details, watch **Video Lessons 2 and 3** by Prof. Barba on YouTube.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('iz22_37mMkk')

In [ ]:
YouTubeVideo('xq9YTcv-fQg')

For a careful walk-through of the discretization of the linear convection equation with finite differences (and also the following steps, up to Step 4), watch **Video Lesson 4** by Prof. Barba on YouTube.

In [ ]:
YouTubeVideo('y2WaK7_iMRI')

## Last but not least

**Remember** to rewrite Step 1 as a fresh Python script or in *your own* Jupyter notebook and then experiment by changing the discretization parameters. Once you have done this, you will be ready for [Step 2](./02_Step_2.ipynb).


***

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

> (The cell above executes the style for this notebook. We modified a style we found on the GitHub of [CamDavidsonPilon](https://github.com/CamDavidsonPilon), [@Cmrn_DP](https://twitter.com/cmrn_dp).)